In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
from itertools import product

# Load data
train_data = pd.read_csv('/content/drive/MyDrive/colab_notebooks/Data/main_dataset.csv')
train_data['Id'] = np.where(train_data['Id'] < 1e-18, 1e-18, train_data['Id'])
train_data['Log_Id'] = np.log10(train_data['Id'])
X = train_data[['Tox', 'Nc', 'Nd', 'Ns', 'Vds', 'Vgs']]
y = train_data['Log_Id']

# Polynomial features and scaling
poly = PolynomialFeatures(degree=3, include_bias=False)
X_poly = poly.fit_transform(X)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_poly)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Define parameter grid
param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'subsample': [0.7, 0.8, 1.0],
    'max_features': [None, 'sqrt', 'log2'],
    'loss': ['squared_error', 'absolute_error', 'huber']
}


KeyboardInterrupt: 

In [ ]:


# DataFrame to store results
results_df = pd.DataFrame(columns=['n_estimators', 'learning_rate', 'max_depth', 'min_samples_split', 
                                   'min_samples_leaf', 'subsample', 'max_features', 'loss', 'R2', 'MAE'])

# Generate all combinations of parameters
param_combinations = list(product(*param_grid.values()))
param_keys = list(param_grid.keys())

# Loop through each parameter combination
for param_values in param_combinations:
    params = dict(zip(param_keys, param_values))

    # Train model with current parameters
    model = GradientBoostingRegressor(random_state=42, **params)
    model.fit(X_train, y_train)

    # Evaluate on the first test set
    y_pred_test = model.predict(X_test)
    r2 = r2_score(y_test, y_pred_test)
    mae = mean_absolute_error(y_test, y_pred_test)
    
    # Load and preprocess the second test data
    test_data = pd.read_csv('/content/drive/MyDrive/colab_notebooks/Data/test_data.csv').iloc[0:203]
    test_data['Id'] = np.where(test_data['Id'] < 1e-18, 1e-18, test_data['Id'])
    test_data['Log_Id'] = np.log10(test_data['Id'])
    X_test_1 = test_data[['Tox', 'Nc', 'Nd', 'Ns', 'Vds', 'Vgs']]
    y_test_1 = test_data['Log_Id']
    X_test_1_transformed = poly.transform(X_test_1)
    X_test_1_scaled = scaler.transform(X_test_1_transformed)

    # Predict and evaluate on the second test dataset
    y_pred_test_1 = model.predict(X_test_1_scaled)
    r2_test_1 = r2_score(y_test_1, y_pred_test_1)
    mae_test_1 = mean_absolute_error(y_test_1, y_pred_test_1)

    # Log scale plot
    plt.figure(figsize=(10, 5))
    plt.plot(X_test_1['Vgs'], y_pred_test_1, color="green", label="Predicted")
    plt.plot(X_test_1['Vgs'], y_test_1, color="blue", label="Actual")
    plt.title(f'Vgs vs Id (Log scale) - Params: {params}')
    plt.xlabel('Vgs')
    plt.ylabel('Log10(Id)')
    plt.legend()
    plt.show()

    # Linear scale plot
    plt.figure(figsize=(10, 5))
    plt.plot(X_test_1['Vgs'], np.maximum(np.power(10, y_pred_test_1), 1e-18), color="green", label="Predicted")
    plt.plot(X_test_1['Vgs'], np.maximum(np.power(10, y_test_1), 1e-18), color="blue", label="Actual")
    plt.title(f'Vgs vs Id (Linear scale) - Params: {params}')
    plt.xlabel('Vgs')
    plt.ylabel('Id')
    plt.legend()
    plt.show()

    # Append results to DataFrame
    current_result = pd.DataFrame({**params, 'R2': r2_test_1, 'MAE': mae_test_1}, index=[0])
    results_df = pd.concat([results_df, current_result], ignore_index=True)

    print("-----------\n--------\n----------\n----------")

# Display results
print(results_df)
